In [1]:
import numpy as np
import pandas as pd

# Add parent directory to Python path
import sys
from pathlib import Path
parent_dir = Path.cwd().parent
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

from nc_csf.models import NCCausalForestDML, NCCausalForestDMLOracle, BaselineCausalForestDML
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('rhc.csv')

print(f"Shape of the dataset: {df.shape}")
print(f"Columns: {list(df.columns)}")
df.head()

Shape of the dataset: (5735, 63)
Columns: ['Unnamed: 0', 'cat1', 'cat2', 'ca', 'sadmdte', 'dschdte', 'dthdte', 'lstctdte', 'death', 'cardiohx', 'chfhx', 'dementhx', 'psychhx', 'chrpulhx', 'renalhx', 'liverhx', 'gibledhx', 'malighx', 'immunhx', 'transhx', 'amihx', 'age', 'sex', 'edu', 'surv2md1', 'das2d3pc', 't3d30', 'dth30', 'aps1', 'scoma1', 'meanbp1', 'wblc1', 'hrt1', 'resp1', 'temp1', 'pafi1', 'alb1', 'hema1', 'bili1', 'crea1', 'sod1', 'pot1', 'paco21', 'ph1', 'swang1', 'wtkilo1', 'dnr1', 'ninsclas', 'resp', 'card', 'neuro', 'gastr', 'renal', 'meta', 'hema', 'seps', 'trauma', 'ortho', 'adld3p', 'urin1', 'race', 'income', 'ptid']


,Unnamed: 0,cat1,cat2,ca,sadmdte,dschdte,dthdte,lstctdte,death,cardiohx,...,meta,hema,seps,trauma,ortho,adld3p,urin1,race,income,ptid
0,1,COPD,NaN,Yes,11142,11151.0,NaN,11382,No,0,...,No,No,No,No,No,0.0,NaN,white,Under $11k,5
1,2,MOSF w/Sepsis,NaN,No,11799,11844.0,11844.0,11844,Yes,1,...,No,No,Yes,No,No,NaN,1437.0,white,Under $11k,7
2,3,MOSF w/Malignancy,MOSF w/Sepsis,Yes,12083,12143.0,NaN,12400,No,0,...,No,No,No,No,No,NaN,599.0,white,$25-$50k,9
3,4,ARF,NaN,No,11146,11183.0,11183.0,11182,Yes,0,...,No,No,No,No,No,NaN,NaN,white,$11-$25k,10
4,5,MOSF w/Sepsis,NaN,No,12035,12037.0,12037.0,12036,Yes,0,...,No,No,No,No,No,NaN,64.0,white,Under $11k,11


I found two papers online that uses RHC dataset. Their ways of choosing covariates X differs a bit, so I created two dataframes to run the test. Note that I didn't run the oracle model since we don't actually know the ground truth.

### Tchetgen Tchetgen, E. J., Ying, A., Cui, Y., Shi, X., and Miao, W. An introduction to proximal causal learning. arXiv preprint arXiv:2009.10982, 2020.

In [3]:
# Treatment A
A_raw = df["swang1"]
if A_raw.dtype == "O":
    A = (A_raw == "RHC").astype(int)
else:
    A = (A_raw.astype(float) > 0).astype(int)

# Outcome Y
Y = df["t3d30"]

# Covariates X
X = pd.DataFrame({
    "age": df["age"],
    "sex": df["sex"],
    "race": df["race"]
})

if X["sex"].dtype == "O":
    X["sex"] = (X["sex"] == "Female").astype(int)

if X["race"].dtype == "O":
    X["race_black"] = (X["race"] == "black").astype(int)
    X = X.drop(columns=["race"])

# Proxies Z & W
Z = df[["pafi1", "paco21"]].copy() 
W = df[["ph1", "hema1"]].copy()  

analysis_cols = pd.concat(
    [
        Y.rename("Y"),
        A.rename("A"),
        X,
        Z.rename(columns={"pafi1": "pafi1", "paco21": "paco21"}),
        W.rename(columns={"ph1": "ph1", "hema1": "hema1"}),
    ],
    axis=1,
)

analysis_df = analysis_cols.dropna().copy()

# Overwrite with cleaned arrays
Y = analysis_df["Y"].values
A = analysis_df["A"].values.astype(int)
X_colnames = [col for col in analysis_df.columns if col not in ["Y", "A", "pafi1", "paco21", "ph1", "hema1"]]
X = analysis_df[X_colnames]
Z = analysis_df[["pafi1", "paco21"]]
W = analysis_df[["ph1", "hema1"]]

print(f"\nFinal X shape: {X.shape}")
analysis_df.head()


Final X shape: (5735, 3)


,Y,A,age,sex,race_black,pafi1,paco21,ph1,hema1
0,30,0,70.25098,0,0,68.00000,40.0,7.359375,58.000000
1,30,1,78.17896,1,0,218.31250,34.0,7.329102,32.500000
2,30,1,46.09198,1,0,275.50000,16.0,7.359375,21.097656
3,30,0,75.33197,1,0,156.65625,30.0,7.459961,26.296875
4,2,1,67.90997,0,0,478.00000,17.0,7.229492,24.000000


In [4]:
X_train, X_test, A_train, A_test, Y_train, Y_test, Z_train, Z_test, W_train, W_test = train_test_split(
    X.values, A, Y, Z.values, W.values, test_size=0.3, random_state=42
)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

Training samples: 4014
Test samples: 1721


In [5]:
baseline = BaselineCausalForestDML(n_estimators=200, min_samples_leaf=20, random_state=42)
baseline.fit_baseline(X_train, A_train, Y_train, verbose=True)
pred_baseline = baseline.effect(X_test).ravel()

print(f"\nBaseline predictions - Mean: {pred_baseline.mean():.4f}, Std: {pred_baseline.std():.4f}")
print(f"Baseline predictions - Min: {pred_baseline.min():.4f}, Max: {pred_baseline.max():.4f}")


Baseline predictions - Mean: -1.8407, Std: 1.8258
Baseline predictions - Min: -7.3242, Max: 1.8520


In [6]:
nccsf = NCCausalForestDML(n_estimators=200, min_samples_leaf=20, cv=5, random_state=42)
nccsf.fit(Y=Y_train, T=A_train, X=X_train, Z=Z_train, W=W_train)
pred_nccsf = nccsf.effect(X_test).ravel()

print(f"\nNC-CSF predictions - Mean: {pred_nccsf.mean():.4f}, Std: {pred_nccsf.std():.4f}")
print(f"NC-CSF predictions - Min: {pred_nccsf.min():.4f}, Max: {pred_nccsf.max():.4f}")


NC-CSF predictions - Mean: -1.0050, Std: 1.9093
NC-CSF predictions - Min: -6.8569, Max: 3.3692


### Sverdrup, E., Cui, Y. Proximal Causal Learning of Conditional Average Treatment Effects

In [7]:
# Treatment A
A_raw = df["swang1"]
if A_raw.dtype == "O":
    A = (A_raw == "RHC").astype(int)
else:
    A = (A_raw.astype(float) > 0).astype(int)

# Outcome Y
Y = df["t3d30"]

# Covariates X
# Note we define cat1_coma and cat2_coma by ourselves since it doesn't exist in the original dataset
# reference: https://search.r-project.org/CRAN/refmans/ATbounds/html/RHC.html
X = pd.DataFrame({
    "age": df["age"],
    "sex": df["sex"],
    "cat1_coma": df["cat1"].apply(lambda x: 1 if x in ["Coma"] else 0),
    "cat2_coma": df["cat2"].apply(lambda x: 1 if x in ["Coma"] else 0),
    "dnr1": df["dnr1"],
    "surv2md1": df["surv2md1"],
    "aps1": df["aps1"],
})

if X["sex"].dtype == "O":
    X["sex"] = (X["sex"] == "Female").astype(int)

if X["dnr1"].dtype == "O":
    X["dnr1"] = X["dnr1"].map({"Yes": 1, "No": 0}).fillna(0).astype(int)

# Proxies Z & W
Z = df[["pafi1", "paco21"]].copy() 
W = df[["ph1", "hema1"]].copy()  

analysis_cols = pd.concat(
    [
        Y.rename("Y"),
        A.rename("A"),
        X,
        Z.rename(columns={"pafi1": "pafi1", "paco21": "paco21"}),
        W.rename(columns={"ph1": "ph1", "hema1": "hema1"}),
    ],
    axis=1,
)

analysis_df = analysis_cols.dropna().copy()

# Overwrite with cleaned arrays
Y = analysis_df["Y"].values
A = analysis_df["A"].values.astype(int)
X_colnames = [col for col in analysis_df.columns if col not in ["Y", "A", "pafi1", "paco21", "ph1", "hema1"]]
X = analysis_df[X_colnames]
Z = analysis_df[["pafi1", "paco21"]]
W = analysis_df[["ph1", "hema1"]]

print(f"\nFinal X shape: {X.shape}")
analysis_df.head()


Final X shape: (5735, 7)


,Y,A,age,sex,cat1_coma,cat2_coma,dnr1,surv2md1,aps1,pafi1,paco21,ph1,hema1
0,30,0,70.25098,0,0,0,0,0.640991,46,68.00000,40.0,7.359375,58.000000
1,30,1,78.17896,1,0,0,0,0.755000,50,218.31250,34.0,7.329102,32.500000
2,30,1,46.09198,1,0,0,0,0.317000,82,275.50000,16.0,7.359375,21.097656
3,30,0,75.33197,1,0,0,0,0.440979,48,156.65625,30.0,7.459961,26.296875
4,2,1,67.90997,0,0,0,1,0.437000,72,478.00000,17.0,7.229492,24.000000


In [8]:
X_train, X_test, A_train, A_test, Y_train, Y_test, Z_train, Z_test, W_train, W_test = train_test_split(
    X.values, A, Y, Z.values, W.values, test_size=0.3, random_state=42
)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

Training samples: 4014
Test samples: 1721


In [9]:
baseline = BaselineCausalForestDML(n_estimators=200, min_samples_leaf=20, random_state=42)
baseline.fit_baseline(X_train, A_train, Y_train, verbose=True)
pred_baseline = baseline.effect(X_test).ravel()

print(f"\nBaseline predictions - Mean: {pred_baseline.mean():.4f}, Std: {pred_baseline.std():.4f}")
print(f"Baseline predictions - Min: {pred_baseline.min():.4f}, Max: {pred_baseline.max():.4f}")


Baseline predictions - Mean: -1.2749, Std: 1.3376
Baseline predictions - Min: -5.9866, Max: 3.0967


In [10]:
nccsf = NCCausalForestDML(n_estimators=200, min_samples_leaf=20, cv=5, random_state=42)
nccsf.fit(Y=Y_train, T=A_train, X=X_train, Z=Z_train, W=W_train)
pred_nccsf = nccsf.effect(X_test).ravel()

print(f"\nNC-CSF predictions - Mean: {pred_nccsf.mean():.4f}, Std: {pred_nccsf.std():.4f}")
print(f"NC-CSF predictions - Min: {pred_nccsf.min():.4f}, Max: {pred_nccsf.max():.4f}")


NC-CSF predictions - Mean: -1.1017, Std: 1.3517
NC-CSF predictions - Min: -5.7010, Max: 3.0812


### Generate semi-syn (use real X, A, Z, W)

**Severity Score Construction for Unmeasured Confounder $U$**

We construct $U$ as a latent severity score from physiological lab values following a Gaussian factor model:

1. **Standardize lab values**: For each lab measurement $L_j$ (where $j \in \{\text{sod1, pot1, crea1, bili1, alb1, pafi1, paco21, ph1, hema1}\}$):
   $$\tilde{L}_j = \frac{L_j - \bar{L}_j}{s_j}$$

2. **Construct weighted score**: Using PCA to obtain weights $w_j$ from the first principal component:
   $$U_i = \sum_{j} w_j \tilde{L}_{ij} + \epsilon_i, \quad \epsilon_i \sim \mathcal{N}(0, \sigma_U^2)$$

3. **Standardize**: Final $U$ is standardized to have mean 0 and variance 1.

This creates a realistic unmeasured confounder that represents underlying patient severity, influencing both treatment assignment and outcomes.

In [27]:
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def generate_semi_synthetic_data(
    analysis_df,
    original_df,
    severity_columns=None,
    use_pca=True,
    sigma_u=0.3,
    target_censor_rate=0.35,
    k_t=1.5,
    lam_t=0.4,
    tau_log_hr=-0.6,
    beta_u_in_t=0.8,
    k_c=1.2,
    beta_u_in_c=0.3,
    seed=123
):
    """
    Generate semi-synthetic survival data using real X, A, Z, W and synthetic Y.
    U is constructed as a severity score from physiological variables (unmeasured confounder).
    
    Parameters:
    -----------
    analysis_df : pd.DataFrame
        DataFrame containing real X, A, Z, W columns (after dropna)
    original_df : pd.DataFrame
        Original DataFrame before dropna (for accessing severity columns)
    severity_columns : list
        List of column names to use for severity score U
        Default: ['sod1', 'pot1', 'crea1', 'bili1', 'alb1', 'pafi1', 'paco21', 'ph1', 'hema1']
    use_pca : bool
        If True, use first principal component for weights; if False, use equal weights
    sigma_u : float
        Standard deviation of noise added to severity score (default: 0.3)
   
    Returns:
    --------
    observed_df : pd.DataFrame
        DataFrame with semi-synthetic observed data (real X,A,Z,W and synthetic time, event)
    truth_df : pd.DataFrame
        DataFrame with additional ground truth columns (U, T0, T1, C0, C1, CATE, etc.)
    """
    
    if severity_columns is None:
        severity_columns = ['sod1', 'pot1', 'crea1', 'bili1', 'alb1', 'pafi1', 'paco21', 'ph1', 'hema1']
    
    rng = np.random.default_rng(seed)
    n = len(analysis_df)
    
    # Extract real data from analysis_df
    A = analysis_df["A"].values.astype(int)
    X_colnames = [col for col in analysis_df.columns if col not in ["Y", "A", "pafi1", "paco21", "ph1", "hema1"]]
    X = analysis_df[X_colnames].values
    Z = analysis_df[["pafi1", "paco21"]].values
    W = analysis_df[["ph1", "hema1"]].values
    
    p = X.shape[1]
    
    # Get indices of analysis_df in original_df to match rows
    # We need to align the rows properly
    analysis_indices = analysis_df.index
    
    # Extract severity columns from original_df using the same indices
    severity_data = original_df.loc[analysis_indices, severity_columns].copy()
    
    # Handle missing values in severity columns (if any remain)
    severity_data = severity_data.fillna(severity_data.mean())
    
    # Standardize the lab values: L_tilde = (L - mean(L)) / std(L)
    scaler = StandardScaler()
    L_tilde = scaler.fit_transform(severity_data)
    
    # Create severity score U (unmeasured confounder)
    if use_pca:
        # Use first principal component direction as weights
        pca = PCA(n_components=1, random_state=seed)
        U_score = pca.fit_transform(L_tilde).ravel()
    else:
        # Use equal weights
        U_score = L_tilde.mean(axis=1)
    
    # Add Gaussian noise: U = weighted_sum + epsilon, epsilon ~ N(0, sigma_u^2)
    epsilon = rng.normal(scale=sigma_u, size=n)
    U = U_score + epsilon
    
    # Standardize U to have mean 0 and std 1 for consistency
    U = (U - U.mean()) / U.std()
    
    # Helper functions from data_generation.py
    def weibull_ph_time(u01, k, lam, eta):
        u01 = np.clip(u01, 1e-12, 1 - 1e-12)
        scale = lam * np.exp(-eta / k)
        return scale * (-np.log(u01)) ** (1.0 / k)
    
    # Generate coefficients for outcome model
    beta_t = rng.normal(scale=0.4, size=p)
    
    # Generate potential event times T0, T1
    u_t = rng.random(n)
    
    # Linear predictor for event times
    eta_t0 = X @ beta_t + beta_u_in_t * U + tau_log_hr * 0.0
    eta_t1 = X @ beta_t + beta_u_in_t * U + tau_log_hr * 1.0
    
    T0 = weibull_ph_time(u_t, k=k_t, lam=lam_t, eta=eta_t0)
    T1 = weibull_ph_time(u_t, k=k_t, lam=lam_t, eta=eta_t1)
    
    # Generate censoring times with calibration
    beta_c = rng.normal(scale=0.3, size=p)
    u_c = rng.random(n)
    eta_c = X @ beta_c + beta_u_in_c * U
    
    # Calibrate censoring to achieve target censoring rate
    T_obs_for_calib = np.where(A == 1, T1, T0)
    
    lo, hi = 1e-8, 1e6
    for _ in range(60):
        mid = 0.5 * (lo + hi)
        C_mid = weibull_ph_time(u_c, k=k_c, lam=mid, eta=eta_c)
        censor_rate_mid = (C_mid < T_obs_for_calib).mean()
        if censor_rate_mid < target_censor_rate:
            hi = mid
        else:
            lo = mid
    lam_c_used = 0.5 * (lo + hi)
    
    C0 = weibull_ph_time(u_c, k=k_c, lam=lam_c_used, eta=eta_c)
    C1 = weibull_ph_time(u_c, k=k_c, lam=lam_c_used, eta=eta_c)
    
    # Realized T, C and observed (time, event)
    T = np.where(A == 1, T1, T0)
    C = np.where(A == 1, C1, C0)
    
    time = np.minimum(T, C)
    event = (T <= C).astype(int)
    
    # Create observed DataFrame (U is NOT included as it's unmeasured)
    observed_df = analysis_df.copy()
    observed_df["time"] = time
    observed_df["event"] = event
    
    # Drop original Y if it exists, replace with new outcome
    if "Y" in observed_df.columns:
        observed_df = observed_df.drop(columns=["Y"])
    
    # Create truth DataFrame with additional ground truth information (including U)
    truth_df = observed_df.copy()
    truth_df.insert(0, "U", U)
    truth_df["T0"] = T0
    truth_df["T1"] = T1
    truth_df["C0"] = C0
    truth_df["C1"] = C1
    truth_df["T"] = T
    truth_df["C"] = C
    truth_df["eta_t0"] = eta_t0
    truth_df["eta_t1"] = eta_t1
    
    # Add ground truth CATE
    # For Weibull PH model: E[T | η] = λ * Γ(1 + 1/k) * exp(-η/k)
    # CATE = E[T(1) - T(0) | X,U] = λ * Γ(1 + 1/k) * (exp(-η₁/k) - exp(-η₀/k))
    G = math.gamma(1.0 + 1.0 / k_t)
    cate_xu = lam_t * G * (np.exp(-eta_t1 / k_t) - np.exp(-eta_t0 / k_t))
    truth_df["CATE_XU"] = cate_xu
    truth_df["ITE"] = T1 - T0
    
    actual_censor_rate = (event == 0).mean()
    print(f"Sample size: {n}")
    print(f"Severity score (U) - Mean: {U.mean():.4f}, Std: {U.std():.4f}")
    print(f"Target censoring rate: {target_censor_rate:.2%}")
    print(f"Actual censoring rate: {actual_censor_rate:.2%}")
    print(f"Treatment proportion: {A.mean():.2%}")
    print(f"Mean CATE: {cate_xu.mean():.4f}")
    
    return observed_df, truth_df

# Generate semi-synthetic dataset with severity score U
observed_df, truth_df = generate_semi_synthetic_data(
    analysis_df=analysis_df,
    original_df=df,
    severity_columns=['sod1', 'pot1', 'crea1', 'bili1', 'alb1', 'pafi1', 'paco21', 'ph1', 'hema1'],
    use_pca=True,
    sigma_u=0.3,
    target_censor_rate=0,
    seed=42
)

print("\nObserved data shape:", observed_df.shape)
print("Columns:", list(observed_df.columns))
print("\nTruth data shape:", truth_df.shape)
print("Columns:", list(truth_df.columns))
print("\nFirst few rows of observed data:")
observed_df.head()

Sample size: 5735
Severity score (U) - Mean: 0.0000, Std: 1.0000
Target censoring rate: 0.00%
Actual censoring rate: 0.05%
Treatment proportion: 38.08%
Mean CATE: 6165861685760.2764

Observed data shape: (5735, 14)
Columns: ['A', 'age', 'sex', 'cat1_coma', 'cat2_coma', 'dnr1', 'surv2md1', 'aps1', 'pafi1', 'paco21', 'ph1', 'hema1', 'time', 'event']

Truth data shape: (5735, 25)
Columns: ['U', 'A', 'age', 'sex', 'cat1_coma', 'cat2_coma', 'dnr1', 'surv2md1', 'aps1', 'pafi1', 'paco21', 'ph1', 'hema1', 'time', 'event', 'T0', 'T1', 'C0', 'C1', 'T', 'C', 'eta_t0', 'eta_t1', 'CATE_XU', 'ITE']

First few rows of observed data:


,A,age,sex,cat1_coma,cat2_coma,dnr1,surv2md1,aps1,pafi1,paco21,ph1,hema1,time,event
0,0,70.25098,0,0,0,0,0.640991,46,68.00000,40.0,7.359375,58.000000,1.996338e+08,1
1,1,78.17896,1,0,0,0,0.755000,50,218.31250,34.0,7.329102,32.500000,1.030332e+10,1
2,1,46.09198,1,0,0,0,0.317000,82,275.50000,16.0,7.359375,21.097656,1.548373e-01,1
3,0,75.33197,1,0,0,0,0.440979,48,156.65625,30.0,7.459961,26.296875,1.247323e+09,1
4,1,67.90997,0,0,0,1,0.437000,72,478.00000,17.0,7.229492,24.000000,1.467039e+04,1
